In [581]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import f1_score
import os
import cv2
import yaml
from ultralytics import YOLO
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout,Layer,Conv2D,InputLayer,MaxPool2D,BatchNormalization,Input
from tensorflow.keras.metrics import BinaryAccuracy,FalsePositives,FalseNegatives,TruePositives,TrueNegatives,Precision,Recall, AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy,SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import  Callback,CSVLogger,EarlyStopping,LearningRateScheduler
from sklearn.metrics import confusion_matrix,roc_curve
from sklearn.preprocessing import LabelEncoder
import pydot
import graphviz
import seaborn as sns
import matplotlib.pyplot as plt


In [582]:

train_dir = 'D:\juvdv2-vdvwc\Train\images'
val_dir = 'D:\juvdv2-vdvwc\Val\images'
train_annotations_dir = 'D:\juvdv2-vdvwc\Train\labels'
val_annotations_dir = 'D:\juvdv2-vdvwc\Val\labels'

In [583]:
def parse_annotation(annotation_dir):
    annotations = []
    for weather in ['Rainny', 'Sunny']:
        for time in ['Day', 'Night']:
            current_annotation_dir = os.path.join(annotation_dir, weather, time)
            for filename in os.listdir(current_annotation_dir):
                if filename.endswith(".txt"):
                    with open(os.path.join(current_annotation_dir, filename)) as f:
                        lines = f.readlines()
                        for line in lines:
                            label = line.split(' ')[0]
                            annotations.append(label)
    return annotations

In [584]:
def load_data(image_dir, annotation_dir):
    images = []
    labels = []
    label_counts = {}
    all_label_lengths = []

    for weather in ['Rainny', 'Sunny']:
        for time in ['Day', 'Night']:
            curr_image_dir = os.path.join(image_dir, weather, time)
            curr_annotation_dir = os.path.join(annotation_dir, weather, time)
            
            for filename in os.listdir(curr_image_dir):
                if filename.endswith(".jpg") or filename.endswith(".jpeg"):
                    image = cv2.imread(os.path.join(curr_image_dir, filename))
                    image = cv2.resize(image, (224, 224))
                    images.append(image)
                    
                    annotation_file = os.path.join(curr_annotation_dir, filename.split('.')[0] + '.txt')
                    if os.path.exists(annotation_file):
                        with open(annotation_file) as f:
                            lines = f.readlines()
                            image_labels = []
                            
                            for line in lines:
                                label = line.split(' ')[0]
                                if label not in ['', '\n', '0.679479']:
                                    image_labels.append(label)
                                    
                                    if label not in label_counts:
                                        label_counts[label] = 1
                                    else:
                                        label_counts[label] += 1

                            labels.append(image_labels)
                            all_label_lengths.append(len(image_labels))  
                    else:
                        labels.append([])
                        all_label_lengths.append(0)

    sorted_label_counts = {k: label_counts[k] for k in sorted(label_counts.keys())}
    return np.array(images), labels, sorted_label_counts, all_label_lengths

In [585]:
def encode_labels(labels):
    all_labels = [item for sublist in labels for item in sublist]
    encoder = LabelEncoder()
    encoder.fit(all_labels)
    
    encoded_labels = [encoder.transform(label) for label in labels]
    return encoded_labels, encoder.classes_

In [586]:
def pad_labels(labels, max_length):
    padded_labels = []
    for label in labels:
        if len(label) < max_length:
            padded_label = padded_label = np.concatenate((label, [-1] * (max_length - len(label))))
        else:
            padded_label = label[:max_length]
        padded_labels.append(padded_label)
    return np.array(padded_labels)

In [587]:
x_train, y_train_labels, train_label_counts, train_label_lengths = load_data(train_dir,train_annotations_dir)
x_val, y_val_labels, val_label_counts, val_label_lengths  = load_data(val_dir,val_annotations_dir)

In [ ]:
y_train_encoded, classes = encode_labels(y_train_labels)
y_val_encoded, _ = encode_labels(y_val_labels)

In [ ]:
max_label_length = max(max(train_label_lengths), max(val_label_lengths))

# Pad labels to the maximum label length
y_train = pad_labels(y_train_encoded, max_label_length)
y_val = pad_labels(y_val_encoded, max_label_length)


In [ ]:
print('Number of images loaded:', len(x_train))
print('Number of labels loaded:', len(y_train))

Number of images loaded: 2600
Number of labels loaded: 2600


In [ ]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((2600, 224, 224, 3), (2600, 42), (200, 224, 224, 3), (200, 42))

In [ ]:
x_train = x_train / 255.0
x_val = x_val / 255.0

In [ ]:
train_annotations = parse_annotation(train_annotations_dir)
val_annotations = parse_annotation(val_annotations_dir)

In [ ]:

print('Train label counts:',train_label_counts)

Train label counts: {'0': 10146, '1': 2379, '10': 9, '11': 132, '12': 2, '13': 166, '14': 49, '2': 1020, '3': 1194, '4': 291, '5': 387, '6': 331, '7': 343, '8': 333, '9': 315}


In [ ]:
class customDense(Layer):
    def __init__(self, output_units, activation=None):
        super(customDense, self).__init__()
        self.output_units = output_units  
        self.activation = activation

    def build(self, input_features_shape):
        self.w = self.add_weight(shape=(input_features_shape[-1], self.output_units),
                                 initializer="random_normal",
                                 trainable=True)  
        
        self.b = self.add_weight(shape=(self.output_units,),
                                 initializer="random_normal",
                                 trainable=True)  

    def call(self, input_features):
        pre_output = tf.matmul(input_features, self.w) + self.b  
        if self.activation == "relu":
            return tf.nn.relu(pre_output)  
        elif self.activation == "sigmoid":
            return tf.math.sigmoid(pre_output)  
        else:
            return pre_output   


In [ ]:
y_val.shape, y_train.shape, x_val.shape, x_train.shape

((200, 42), (2600, 42), (200, 224, 224, 3), (2600, 224, 224, 3))

In [ ]:
IM_SIZE = 224
def resize_rescale(image):
    resized_image = cv2.resize(image, (IM_SIZE,IM_SIZE))
    rescaled_image = resized_image / 255.0
    return rescaled_image
    

In [ ]:
for i in range(len(x_train)):
    x_train[i] = resize_rescale(x_train[i])

for i in range(len(x_val)):
    x_val[i] = resize_rescale(x_val[i]) 

In [ ]:

IM_SIZE =224
lenet_custom_model = tf.keras.Sequential([
    InputLayer(input_shape = (IM_SIZE,IM_SIZE,3)),
    Conv2D(
        filters = 6,
        kernel_size = 5,
        strides=(1,1),
        padding ='valid',
        activation = 'relu'  
    ),
    BatchNormalization(),
    MaxPool2D(
        pool_size = 2,
        strides = 2
    ),

    Conv2D(
        filters = 16,
        kernel_size = 5,
        strides=(1,1),
        padding ='valid',
        activation = 'relu'  
    ),
    BatchNormalization(),
    MaxPool2D(
        pool_size = 2,
        strides = 2
    ),

    Flatten(),

    customDense(100,activation="relu"),
    BatchNormalization(),
    
    customDense(10,activation="relu"), 
    BatchNormalization(),
    customDense(1,activation="sigmoid"),
])

c:\Users\chaga\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
lenet_custom_model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_35 (Conv2D)              │ (None, 220, 220, 6)    │           456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_67          │ (None, 220, 220, 6)    │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_32 (MaxPooling2D) │ (None, 110, 110, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 106, 106, 16)   │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_68          │ (None, 106, 106, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_33 (MaxPooling2D) │ (None, 53, 53, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_16 (Flatten)            │ (None, 44944)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_dense_48 (customDense)   │ (None, 100)            │     4,494,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_69          │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_dense_49 (customDense)   │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_70          │ (None, 10)             │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_dense_50 (customDense)   │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,498,921 (17.16 MB)

 Trainable params: 4,498,657 (17.16 MB)

 Non-trainable params: 264 (1.03 KB)

In [ ]:
metrics = [TruePositives(name ='tp'),TrueNegatives(name='tn'),FalsePositives(name='fp'),FalseNegatives(name='fn'),
           Precision(name='precision'),Recall(name='recall'),AUC(name='auc'),BinaryAccuracy(name='accuracy')]

In [ ]:
def scheduler(epoch,lr):
    if epoch<=10:
        return lr
    else:
        return (lr*tf.math.exp(-0.1)).numpy()
scheduler_callback = LearningRateScheduler(scheduler,verbose=1)

In [ ]:
lenet_custom_model.compile(
    optimizer=Adam(learning_rate = 0.01),
    loss = SparseCategoricalCrossentropy(),
    metrics = metrics
)

In [ ]:
history = lenet_custom_model.fit(x_train, y_train, batch_size=16, epochs=50, validation_data=(x_val, y_val),callbacks=[scheduler_callback])

MemoryError: Unable to allocate 1.46 GiB for an array with shape (2600, 224, 224, 3) and data type float32